# Import

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os,sys
from IPython.display import Markdown
from importlib import reload

# Load Data

In [2]:
data = pd.read_csv('telva_dataset.csv', header=0,sep=';')
data= data.drop('date',axis=1)
data= data.drop('water_volume_variation',axis=1)
display(data.head(3))
print('size : ',data.shape)
print('Missing Data : ',data.isna().sum().sum())

,water_precipitation,water_height,water_flow,water_volume
0,0.0,31.87,12.2,11983985.0
1,2.2,31.85,11.0,11932800.0
2,2.2,31.84,10.7,11907258.0


size :  (10227, 4)
Missing Data :  0


# Split Data

In [3]:
X= data.drop('water_flow',axis=1)
y=data['water_flow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8181, 3) (2046, 3) (8181,) (2046,)


# Normalize Data

In [4]:
display(X_train.describe().style.format("{0:.2f}").set_caption("Before normalization :"))

mean = X_train.mean()
std  = X_train.std()
X_train = (X_train - mean) / std
X_test  = (X_test  - mean) / std

display(X_train.describe().style.format("{0:.2f}").set_caption("After normalization :"))

# Convert ou DataFrame to numpy array
X_train, y_train = np.array(X_train), np.array(y_train)
X_test,  y_test  = np.array(X_test),  np.array(y_test)

,water_precipitation,water_height,water_volume
count,8181.00,8181.00,8181.00
mean,3.13,32.52,14096957.97
std,6.90,1.47,3899472.86
min,0.00,28.23,4931058.00
25%,0.00,31.78,11754698.00
50%,0.00,32.59,13916032.00
75%,3.00,33.65,17085322.00
max,105.00,35.15,21687518.00


,water_precipitation,water_height,water_volume
count,8181.00,8181.00,8181.00
mean,-0.00,-0.00,-0.00
std,1.00,1.00,1.00
min,-0.45,-2.93,-2.35
25%,-0.45,-0.51,-0.60
50%,-0.45,0.05,-0.05
75%,-0.02,0.77,0.77
max,14.77,1.79,1.95


# Build DNN model with keras

In [5]:
def get_model_v1(shape):
 model = keras.models.Sequential()
 model.add(keras.layers.Input(shape, name="InputLayer"))
 model.add(keras.layers.Dense(64, activation='relu', name='Dense_n1'))
 model.add(keras.layers.Dense(64, activation='relu', name='Dense_n2'))
 model.add(keras.layers.Dense(1, name='Output'))
 model.compile(optimizer='rmsprop', loss='mse', metrics=['mae','mse'])
 return model

model=get_model_v1( (3,) )
model.summary()
os.makedirs('./run/models',   mode=0o750, exist_ok=True)
save_dir = "./run/models/best_model.h5"
savemodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, save_best_only=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_n1 (Dense)            (None, 64)                256       
                                                                 
 Dense_n2 (Dense)            (None, 64)                4160      
                                                                 
 Output (Dense)              (None, 1)                 65        
                                                                 
Total params: 4,481
Trainable params: 4,481
Non-trainable params: 0
_________________________________________________________________


# Train and save the DNN model

In [6]:
history = model.fit(X_train, y_train, epochs= 100, batch_size= 10, validation_data = (X_test, y_test), callbacks= [savemodel_callback])
model.save('my_model.h5')
model = load_model('my_model.h5')

Epoch 1/100
819/819 [==============================] - 2s 2ms/step - loss: 35.1834 - mae: 3.5262 - mse: 35.1834 - val_loss: 39.5787 - val_mae: 3.6839 - val_mse: 39.5787
Epoch 2/100
819/819 [==============================] - 1s 2ms/step - loss: 32.7522 - mae: 3.4773 - mse: 32.7522 - val_loss: 39.2456 - val_mae: 3.4379 - val_mse: 39.2456
Epoch 3/100
819/819 [==============================] - 1s 2ms/step - loss: 32.5404 - mae: 3.4578 - mse: 32.5404 - val_loss: 39.2860 - val_mae: 3.3713 - val_mse: 39.2860
Epoch 4/100
819/819 [==============================] - 1s 2ms/step - loss: 32.4673 - mae: 3.4540 - mse: 32.4673 - val_loss: 38.6766 - val_mae: 3.5984 - val_mse: 38.6766
Epoch 5/100
819/819 [==============================] - 1s 2ms/step - loss: 32.3959 - mae: 3.4425 - mse: 32.3959 - val_loss: 38.8819 - val_mae: 3.6653 - val_mse: 38.8819
Epoch 6/100
819/819 [==============================] - 1s 2ms/step - loss: 32.3767 - mae: 3.4475 - mse: 32.3767 - val_loss: 38.5615 - val_mae: 3.7070 - val

# Evaluate the model

In [7]:
score = model.evaluate(X_test, y_test, verbose=0)
print('x_test / loss      : {:5.4f}'.format(score[0]))
print('x_test / mae       : {:5.4f}'.format(score[1]))
print('x_test / mse       : {:5.4f}'.format(score[2]))
print("min( val_mae ) : {:.4f}".format( min(history.history["val_mae"]) ) )

x_test / loss      : 38.9221
x_test / mae       : 3.6129
x_test / mse       : 38.9221
min( val_mae ) : 3.2378


# Make and display prediction

In [8]:
y_pred = model.predict(X_test)
print(y_pred)

64/64 [==============================] - 0s 1ms/step
[[9.031281 ]
 [2.9480765]
 [4.2165117]
 ...
 [5.1594534]
 [8.132318 ]
 [4.221397 ]]


In [9]:
n = 200
ii = np.random.randint(1,len(X_test),n)
x_sample = X_test[ii]
y_sample = y_test[ii]
print('Capture    Prediction   Real   Delta')
for i in range(n):
    pred   = y_pred[i][0]
    real   = y_test[i]
    delta  = real-pred
    print(f'{i:03d}        {pred:.2f}       {real}      {delta:+.2f} ')

Capture    Prediction   Real   Delta
000        9.03       32.97      +23.94 
001        2.95       1.54      -1.41 
002        4.22       2.5      -1.72 
003        8.93       2.13      -6.80 
004        6.06       1.7      -4.36 
005        4.08       6.19      +2.11 
006        6.16       1.71      -4.45 
007        3.38       1.61      -1.77 
008        4.59       1.85      -2.74 
009        3.14       1.48      -1.66 
010        5.08       1.37      -3.71 
011        4.22       14.0      +9.78 
012        4.64       2.19      -2.45 
013        5.10       11.38      +6.28 
014        7.41       12.24      +4.83 
015        5.74       7.82      +2.08 
016        3.58       1.37      -2.21 
017        5.77       4.08      -1.69 
018        3.71       2.19      -1.52 
019        7.33       1.93      -5.40 
020        3.81       1.54      -2.27 
021        4.36       2.64      -1.72 
022        3.58       1.57      -2.01 
023        3.98       2.41      -1.57 
024        4.51       6.8